# 11번가 유아식 Crawling

In [1]:
from selenium import webdriver as wd
from bs4 import BeautifulSoup as bs
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## Step1 
- best100 link(seed_url) 가져오기

In [4]:
main_url = "http://www.11st.co.kr/browsing/BestSeller.tmall?method=getBestSellerMain&dispCtgrNo=1001574"

In [44]:
# 드라이버 로드
driver = wd.Chrome(executable_path='chromedriver.exe')

In [45]:
driver.get(main_url)

In [46]:
html = bs(driver.page_source, "lxml")

In [8]:
html

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<html class="fiexible_resize" lang="ko" xmlns="http://www.w3.org/1999/xhtml"><head><script async="" src="http://www.google-analytics.com/plugins/ua/ec.js" type="text/javascript"></script><script async="" src="http://www.google-analytics.com/analytics.js" type="text/javascript"></script><script async="" src="https://rake.skplanet.com:443/log/static/js/rake/current/rake.bundle.js" type="text/javascript"></script>
<title>11번가 베스트 - 11번가</title>
<meta content="Upgrade Your Life - 11번가" name="description"/>
<meta content="Upgrade Your Life - 11번가" name="keywords"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<script async="" src="//www.googletagmanager.com/gtm.js?id=GTM-P3PMQD&amp;l=GA_dataLayer"></script><script async="" src="//www.google-analytics.com/analytics.js"></script><script src="//c.011st.com/html/exp

In [47]:
# elements -> list, element -> webElement
links = driver.find_elements_by_css_selector('.pup_title a')
type(links)

list

In [48]:
seed_url = []
for link in links:
    link = link.get_attribute('href')
    seed_url.append(link)
print(len(seed_url))

100


---

## step 2
- link 하나하나 들어가서 정보 가져오기 
- 필요한 정보 
    * Document_id
    * Category
    * Url
    * Title
    * Price
    * Photo
    * CountSell
    * CountRiple
    * Seller
    * Riple

In [49]:
len(seed_url)

100

In [50]:
type(seed_url)

list

In [52]:
seed_url[:5]

['http://www.11st.co.kr/product/SellerProductDetail.tmall?method=getSellerProductDetail&prdNo=1234985926&trTypeCd=22&trCtgrNo=895019',
 'http://www.11st.co.kr/product/SellerProductDetail.tmall?method=getSellerProductDetail&prdNo=1046705025&trTypeCd=22&trCtgrNo=895019',
 'http://www.11st.co.kr/product/SellerProductDetail.tmall?method=getSellerProductDetail&prdNo=1688098477&trTypeCd=22&trCtgrNo=895019',
 'http://www.11st.co.kr/product/SellerProductDetail.tmall?method=getSellerProductDetail&prdNo=617652955&trTypeCd=22&trCtgrNo=895019',
 'http://www.11st.co.kr/product/SellerProductDetail.tmall?method=getSellerProductDetail&prdNo=617652356&trTypeCd=22&trCtgrNo=895019']

In [58]:
seed_url[0]

'http://www.11st.co.kr/product/SellerProductDetail.tmall?method=getSellerProductDetail&prdNo=1234985926&trTypeCd=22&trCtgrNo=895019'

### 해결할 것 ) 정규식 

In [53]:
import re

In [85]:
re.sub(r'[^prdNo=\d{10}]','',seed_url[0])

'p11orprodrrodod=rrodprdNo=1234985926rpd=22rrNo=895019'

---

In [86]:
page = driver.get(seed_url[0])

In [87]:
content = bs(driver.page_source, 'lxml')

In [89]:
content

<!DOCTYPE html>
<html lang="ko" xmlns="http://www.w3.org/1999/xhtml"><head><script async="" src="http://www.google-analytics.com/plugins/ua/ec.js" type="text/javascript"></script><script async="" src="http://www.google-analytics.com/analytics.js" type="text/javascript"></script><script async="" src="https://rake.skplanet.com:443/log/static/js/rake/current/rake.bundle.js" type="text/javascript"></script><script async="" src="//www.googletagmanager.com/gtm.js?id=GTM-P3PMQD&amp;l=GA_dataLayer"></script><script async="" src="//www.google-analytics.com/analytics.js"></script>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=edge, chrome=1" http-equiv="X-UA-Compatible"/>
<title>[내아이애]아기부터 어른까지 유기농 과자 특가(10+2) - 11번가</title>
<meta content="대표 안심 쌀과자/떡뻥 특가 할인 이벤트!!, 카테고리 : 유아동 과자, 가격 : 1500원" name="description"/>
<meta content="Upgrade Your Life - 11번가" name="keywords"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE

##### category 

In [92]:
category = driver.find_element_by_css_selector('#headSel_3')

In [94]:
category = category.text
print(category)

유아식


##### Title

In [95]:
title = driver.find_element_by_css_selector('.heading h2')

In [98]:
title = title.text
print(title)

[내아이애]아기부터 어른까지 유기농 과자 특가(10+2)


##### Price
- 기본 price
- 옵션 선택에서 max price 찾아서 '최소~최대' 로 나타내줘야할까?  
- 옵션에 있는 상품도 가져와 최저가 비교 (이렇게 해야 레알 최저가 비교 아닐까?)
- 품절인거 빼야함 --> 알아서 blocking 되있기 때문에 별도의 처리 x 

In [99]:
price = driver.find_element_by_css_selector('.price_detail strong').text
print(price)

1,500


In [110]:
options = driver.find_elements_by_css_selector('.ui_option_list .prdc_price em')

In [111]:
len(options)

51

In [117]:
# 옵션 첫번째 값 확인
a = options[0].text
re.sub(r'\W','',a)

'1980'

In [119]:
# 옵션 값 중에서 가장 큰 값 
maxPrice = 0
for option in options:
    option = option.text
    option = int(re.sub(r'\W','',option))
    if maxPrice < option:
        maxPrice = option

In [120]:
maxPrice

2890

##### Photo 
- base64